In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta

# IMPORT CSV
csv = pd.read_csv('https://raw.githubusercontent.com/yellowslides/charts/main/2022charts.csv')

# GLOBAL VARIABLES
chartNumbers = csv.chart.unique()
startDate = date(2022, 1, 6)

# CREATE DICTIONARY OF 2022 ALBUMS
twentyTwoChartsDict = {elem : pd.DataFrame() for elem in chartNumbers}
for key in twentyTwoChartsDict.keys():
    twentyTwoChartsDict[key] = csv[:][csv.chart == key]

# RETURN CHART BY CHART CODE  
def chartByChartCode(chartCode):
  return twentyTwoChartsDict[chartCode]

# RETURN DATE OF CHART BY CHART CODE AS STRING
def dateByChartCode(chartCode):
  return (startDate + timedelta(weeks=(int((chartCode)-1)%100))).strftime('%m/%d/%Y') 

#for i in range(2201,2253):
#  print(dateByChartNumber(i))

# RETURN LIST OF DATES OF CHARTS BY CHART CODES
def datesByChartNumbers(listOfCodes):
  listOfDates = []
  for i in listOfCodes:
    listOfDates.append(dateByChartCode(i))
  return listOfDates

#datesByChartNumbers(chartNumbers)

# INSERT LIST OF DATES TO A DATAFRAME BY CHART CODES
def insertDateList(df, listOfCodes):
  return df.insert(0, 'date', datesByChartNumbers(listOfCodes))

# RESET INDEX ON A DATAFRAME
def resetIndex(df):
  df.insert(0, '#', np.arange(1, len(df)+1)) 
  df.set_index('#', inplace=True)
  return df
  

In [2]:
# RETURN DATAFRAME OF #1 SONGS
def numberOnes():
  df = csv[csv.position == 1].filter(['song', 'artist'])
  insertDateList(df, chartNumbers)
  return resetIndex(df)

numberOnes()

,date,song,artist
#,,,
1,01/06/2022,Girl,Jamie xx
2,01/13/2022,Sacrifice,The Weeknd
3,01/20/2022,oh my love,FKA twigs
4,01/27/2022,Light Switch,Charlie Puth
5,02/03/2022,Pedestrian at Best,Courtney Barnett
6,02/10/2022,SAOKO,ROSALÍA
7,02/17/2022,Too Dead Inside,yeule
8,02/24/2022,HANN (Alone),(G)I-DLE
9,03/03/2022,Baby,Charli XCX


In [3]:
# RETURN DATAFRAME OF TOP *NUM* SONGS ON CHART IN WEEKS
def mostWeeks(num):
  df = csv[['song','artist']].value_counts()[:num].reset_index()
  df.columns = ['song', 'artist', 'weeks']
  return resetIndex(df)

mostWeeks(10)

,song,artist,weeks
#,,,
1,Girl,Jamie xx,10
2,Send It Up,Kanye West,8
3,On Sight,Kanye West,6
4,Happy New Year,Let's Eat Grandma,5
5,The Reeling,Passion Pit,5
6,Blood On The Leaves,Kanye West,4
7,Gasoline,The Weeknd,4
8,Chaeri,Magdalena Bay,4
9,SAOKO,ROSALÍA,4


In [4]:
# RETURN LENGTH OF LONGEST SEQUENCE OF ASCENDING NUMBERS IN A LIST
def longestAscendingSequence(list):
  currentSequence = 1
  maxSequence = 1
  currentNumber = 0
  maxNumber = 0
  for i in range(1, len(list)):
    if (list[i-1] == list[i]-1):
      currentSequence += 1
      currentNumber = list[i]      
    else:
      currentSequence = 1
    if (currentSequence > maxSequence):
      maxSequence = currentSequence
      maxNumber = currentNumber
  return (maxSequence, maxNumber)

#print(longestAscendingSequence([1,2,3,4,5,6]))   # return 6
#print(longestAscendingSequence([1,2,3,4,6]))     # return 4
#print(longestAscendingSequence([1,3,4,5,6]))     # return 4
#print(longestAscendingSequence([1,3,5]))         # return 1

# RETURN DATAFRAME OF TOP *NUM* SONGS WITH MOST CONSECUTIVE WEEKS ON CHART
def mostConWeeks(num):
  df = mostWeeks(num*2)
  listOfWeeks = []
  for index, row in df.iterrows():
    conWeeks, lastDate = longestAscendingSequence((csv.loc[(csv['song'] == row['song']) 
        & (csv['artist'] == row['artist'])])['chart'].tolist())
    df.iloc[index - 1, 2] = conWeeks
    listOfWeeks.append(dateByChartCode(lastDate-(conWeeks-1)) + " - " + dateByChartCode(lastDate))
  df['dates of run'] = listOfWeeks
  return resetIndex(df.nlargest(num, 'weeks'))

mostConWeeks(10)

,song,artist,weeks,dates of run
#,,,,
1,Girl,Jamie xx,8,01/06/2022 - 02/24/2022
2,Gasoline,The Weeknd,4,01/13/2022 - 02/03/2022
3,Chaeri,Magdalena Bay,4,02/10/2022 - 03/03/2022
4,SAOKO,ROSALÍA,4,02/10/2022 - 03/03/2022
5,Send It Up,Kanye West,3,01/06/2022 - 01/20/2022
6,On Sight,Kanye West,3,02/24/2022 - 03/10/2022
7,Happy New Year,Let's Eat Grandma,3,01/06/2022 - 01/20/2022
8,Domino,Magdalena Bay,3,01/20/2022 - 02/03/2022
9,What It Is,Amber Mark,3,02/03/2022 - 02/17/2022


In [5]:
# RETURN DATAFRAME OF TOP *NUM* SONGS WITH MOST WEEKS IN TOP TEN
def mostTopTenWeeks(num):
  df = csv[csv.position <= 10].filter(['song', 'artist']).value_counts()[:num].reset_index()
  df.columns = ['song', 'artist', 'weeks']
  return resetIndex(df)

mostTopTenWeeks(10)

,song,artist,weeks
#,,,
1,Girl,Jamie xx,8
2,Send It Up,Kanye West,3
3,Baby,Charli XCX,3
4,What It Is,Amber Mark,2
5,HANN (Alone),(G)I-DLE,2
6,Head On Fire,"Griff, Sigrid",2
7,Foreign Things,Amber Mark,2
8,Robot Rock,Daft Punk,2
9,Feel Special,TWICE,2


In [6]:
# RETURN DATAFRAME OF TOP *NUM* SONGS WITH MOST WEEKS AT NUMBER ONE
#def mostWeeksAtNumberOne(num):
#  df = numberOnes()[['title','artist']].value_counts()[:num].reset_index()
#  df.columns = ['song', 'artist', 'weeks']
#  return resetIndex(df)

#mostWeeksAtNumberOne(10)

In [10]:
# RETURN CHART PERFORMANCE FOR A GIVEN SONG BY ARTIST
def chartRun(s, a):
  return csv.loc[(csv['song'] == s) & (csv['artist'] == a)].position.tolist()

girlRun = chartRun('Girl', 'Jamie xx')  
gasolineRun = (chartRun('Gasoline', 'The Weeknd'))

print(girlRun)
print(gasolineRun)

# RETURN IF CHART PERFORMANCE HAS A TOP TEN PEAK
def topTen(chartRun):
  for position in chartRun:
    if position < 11:
      return True
  return False

#topTen(girlRun)
#topTen(gasolineRun)

[1, 2, 9, 4, 11, 7, 7, 6, 21, 7]
[15, 13, 32, 38]


In [11]:
# RETURN DATAFRAME OF TOP *NUM* SONGS WITH MOST WEEKS ON CHART WITHOUT PEAKING TOP TEN
#def mostWeeksWithoutTopTenPeak(num):
#  songsWithTopTenPeaks = mostTopTenWeeks(100)
#  for index, row in songsWithTopTenPeaks:
#    #df = csv[csv.song != row['song']]
#  return df

#mostWeeksWithoutTopTenPeak(100)

In [40]:
# RETURN ARTISTS WITH MOST SONGS AT NUMBER ONE
def mostArtistNumberOnes():
  df = numberOnes()['artist'].value_counts()[:10].reset_index()
  return resetIndex(df)

#print(mostArtistNumberOnes())

# RETURN ARTISTS WITH MOST SONGS AT NUMBER ONE
#def mostArtistTopTens():
#  df = csv[csv.position <= 10].filter(['song', 'artist']).value_counts()[:num].reset_index()

# RETURN ALL SONGS FOR A GIVEN ARTIST
def artistSongs(a):
  li = csv.loc[(csv['artist'] == a)]['song'].drop_duplicates().to_list()
  return li

print(artistSongs('Taylor Swift'))
print(artistSongs('Charli XCX'))
print(artistSongs('Ariana Grande'))
print(artistSongs('Lana Del Rey'))
print(artistSongs('The Weeknd'))

#

["Red (Taylor's Version)", 'Karma', 'Lavender Haze', 'Midnight Rain', 'Bejeweled']
['Come to My Party', 'Baby', '5 in the Morning', '7 years', 'Focus', 'detonate', 'pink diamond', 'Yuck']
['better off', 'goodnight n go', 'in my head', 'R.E.M.', 'Sometimes', 'positions']
[]
[]
['Sacrifice', 'Best Friends', 'Gasoline', 'How Do I Make You Love Me?', 'Take My Breath', 'Is There Someone Else?', 'Out of Time', 'Wasted Times', 'House of Balloons / Glass Table Girls']
